**Imports**

In [ ]:
import pandas as pd
import pickle

**Useful Functions**

In [ ]:
def ratios_data_processing(file_name):
    try:
        df = pd.read_csv(filepath_or_buffer=file_name, index_col=0)
        if 'ticker' in df.columns:
            df.set_index('ticker', inplace=True)
        df.drop('ttm', inplace=True, axis=1)
        prefix = '_'
        df = df.add_prefix(prefix)
        return df
    except:
        print('An error ocurred. Check the file name or its path')
        return 0

In [ ]:
def companies_data_processing(file_name):
    try:
        with open(file_name,'rb') as file:
            raw_data = file.read()
            dict_companies = pickle.loads(raw_data)
        return dict_companies
    except:
        print('An error ocurred. Check the file name or its path')
        return 0

In [ ]:
#Load the csv and pickle files

def data_loading(*file_names):
    
    loaded_data = []
    for name in file_names:
        if name[-3:] == 'csv':
            loaded_data.append(ratios_data_processing(name))
        elif name[-6:] == 'pickle':
            loaded_data.append(companies_data_processing(name))
        else:
            continue
    
    return loaded_data

In [ ]:
#Get sector and industry for each ticker

def get_company_sectors_and_industries(df, dict_companies):
    sector_dict = dict()
    industry_dict = dict()
    for ticker in df.index:
        company_profile = dict_companies[ticker]['_Company__profile']
        sector_dict[ticker] = company_profile.loc['sector', 0]
        industry_dict[ticker] = company_profile.loc['industry', 0]

    #Convert the dictionaries to dataframe to ease the analysis
    sectors_df = pd.DataFrame(list(sector_dict.items()), columns=['ticker', 'sector']).set_index('ticker')
    industries_df = pd.DataFrame(list(industry_dict.items()), columns=['ticker', 'industry']).set_index('ticker')
    
    return sectors_df, industries_df

In [ ]:
def get_sectors_and_industries(df, dict_companies):
    sectors_df, industries_df = get_company_sectors_and_industries(df, dict_companies)
    
    sectors = list(sectors_df.sector.unique())
    industries = list(industries_df.industry.unique())

    return sectors, industries

In [ ]:
#Get only ratios of companies that belong to the specified sector

def get_sector_companies(df, dict_companies, sector):
    sectors_df, industries_df = get_company_sectors_and_industries(df, dict_companies)
    sector_df = df.loc[list(sectors_df.loc[sectors_df.sector == sector].index)]
    
    return sector_df